# Data Types - RDD-based API

> https://spark.apache.org/docs/latest/mllib-data-types.html#local-vector

## local vectors: dense and sparse.

- A dense vector 
is backed by a double array representing its entry values

- sparse vector i
s backed by two parallel arrays: indices and values. 

For example, a vector (1.0, 0.0, 3.0) can be represented in dense format as [1.0, 0.0, 3.0] or in sparse format as (3, [0, 2], [1.0, 3.0]), where 3 is the size of the vector.

### dense vector 
- NumPy’s array
- Python’s list, e.g., [1, 2, 3]

In [4]:
import numpy as np
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors

# Use a NumPy array as a dense vector.
dv1 = np.array([1.0, 0.0, 3.0])
# Use a Python list as a dense vector.
dv2 = [1.0, 0.0, 3.0]

print(dv1)
print(dv2)


### Sparse Vector

- MLlib’s SparseVector.
- SciPy’s csc_matrix with a single column

In [6]:
# Create a SparseVector.
sv1 = Vectors.sparse(3, [0, 2], [1.0, 3.0])
# Use a single-column SciPy csc_matrix as a sparse vector.
sv2 = sps.csc_matrix((np.array([1.0, 3.0]), np.array([0, 2]), np.array([0, 2])), shape=(3, 1))

print(sv1)
print(sv2)

## Labeled point

- A labeled point is a local vector, either dense or sparse, associated with a label/response. 
- In MLlib, labeled points are used in supervised learning algorithms. 
- For binary classification, a label should be either 0 (negative) or 1 (positive). 
- For multiclass classification, labels should be class indices starting from zero: 0, 1, 2, ....

In [8]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

# Create a labeled point with a positive label and a dense feature vector.
pos = LabeledPoint(1.0, [1.0, 0.0, 3.0])

# Create a labeled point with a negative label and a sparse feature vector.
neg = LabeledPoint(0.0, SparseVector(3, [0, 2], [1.0, 3.0]))

print(pos)
print(neg)

## Local matrix
- A local matrix has integer-typed row and column indices and double-typed values, stored on a single machine. 
- MLlib supports :
> - dense matrices, whose entry values are stored in a single double array in column-major order, and 
> - sparse matrices, whose non-zero entry values are stored in the Compressed Sparse Column (CSC) format in column-major order. 
- For example, the following dense matrix
> - [1.0 2.0
> - 3.0 4.0
> - 5.0 6.0]

is stored in a one-dimensional array [1.0, 3.0, 5.0, 2.0, 4.0, 6.0] with the matrix size (3, 2).

In [10]:
from pyspark.mllib.linalg import Matrix, Matrices

# Create a dense matrix ((1.0, 2.0), (3.0, 4.0), (5.0, 6.0))
dm2 = Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])
print(dm2)

# Create a sparse matrix ((9.0, 0.0), (0.0, 8.0), (0.0, 6.0))
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])
print(sm)

##Distributed matrix
- A distributed matrix has long-typed row and column indices and double-typed values, stored distributively in one or more RDDs. 
- Converting a distributed matrix to a different format may require a global shuffle, which is quite expensive. 
- Four types of distributed matrices have been implemented so far.
> - 1. The basic type is called RowMatrix. A RowMatrix is a row-oriented distributed matrix without meaningful row indices, e.g., a collection of feature vectors. It is backed by an RDD of its rows, where each row is a local vector. We assume that the number of columns is not huge for a RowMatrix so that a single local vector can be reasonably communicated to the driver and can also be stored / operated on using a single node. 
> - 2 . An IndexedRowMatrix is similar to a RowMatrix but with row indices, which can be used for identifying rows and executing joins. 
> - 3 . A CoordinateMatrix is a distributed matrix stored in coordinate list (COO) format, backed by an RDD of its entries. 
> - 4 . A BlockMatrix is a distributed matrix backed by an RDD of MatrixBlock which is a tuple of (Int, Int, Matrix).

## RowMatrix
- A RowMatrix is a row-oriented distributed matrix without meaningful row indices, backed by an RDD of its rows, where each row is a local vector. 
- Since each row is represented by a local vector, the number of columns is limited by the integer range but it should be much smaller in practice.

In [13]:
from pyspark.mllib.linalg.distributed import RowMatrix

# Create an RDD of vectors.
rows = sc.parallelize([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])

# Create a RowMatrix from an RDD of vectors.
mat = RowMatrix(rows)

# Get its size.
m = mat.numRows()  # 4
n = mat.numCols()  # 3

# Get the rows as an RDD of vectors again.
rowsRDD = mat.rows

def g(x):
      print x
    
rowsRDD.foreach(g)

      

## IndexedRowMatrix
- An IndexedRowMatrix is similar to a RowMatrix but with meaningful row indices. 
- It is backed by an RDD of indexed rows, so that each row is represented by its index (long-typed) and a local vector.

In [15]:
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix

# Create an RDD of indexed rows.
#   - This can be done explicitly with the IndexedRow class:
indexedRows = sc.parallelize([IndexedRow(0, [1, 2, 3]),
                              IndexedRow(1, [4, 5, 6]),
                              IndexedRow(2, [7, 8, 9]),
                              IndexedRow(3, [10, 11, 12])])
#   - or by using (long, vector) tuples:
indexedRows = sc.parallelize([(0, [1, 2, 3]), (1, [4, 5, 6]),
                              (2, [7, 8, 9]), (3, [10, 11, 12])])

# Create an IndexedRowMatrix from an RDD of IndexedRows.
mat = IndexedRowMatrix(indexedRows)

# Get its size.
m = mat.numRows()  # 4
n = mat.numCols()  # 3

# Get the rows as an RDD of IndexedRows.
rowsRDD = mat.rows

# Convert to a RowMatrix by dropping the row indices.
rowMat = mat.toRowMatrix()
print(rowMat)

## CoordinateMatrix
- A CoordinateMatrix is a distributed matrix backed by an RDD of its entries. 
- Each entry is a tuple of (i: Long, j: Long, value: Double), where i is the row index, j is the column index, and value is the entry value. 
- A CoordinateMatrix should be used only when both dimensions of the matrix are huge and the matrix is very sparse.

In [17]:
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry

# Create an RDD of coordinate entries.
#   - This can be done explicitly with the MatrixEntry class:
entries = sc.parallelize([MatrixEntry(0, 0, 1.2), MatrixEntry(1, 0, 2.1), MatrixEntry(6, 1, 3.7)])
#   - or using (long, long, float) tuples:
entries = sc.parallelize([(0, 0, 1.2), (1, 0, 2.1), (2, 1, 3.7)])

# Create an CoordinateMatrix from an RDD of MatrixEntries.
mat = CoordinateMatrix(entries)

# Get its size.
m = mat.numRows()  # 3
n = mat.numCols()  # 2

# Get the entries as an RDD of MatrixEntries.
entriesRDD = mat.entries

# Convert to a RowMatrix.
rowMat = mat.toRowMatrix()

# Convert to an IndexedRowMatrix.
indexedRowMat = mat.toIndexedRowMatrix()

# Convert to a BlockMatrix.
blockMat = mat.toBlockMatrix()

## BlockMatrix
- A BlockMatrix is a distributed matrix backed by an RDD of MatrixBlocks, 
- where a MatrixBlock is a tuple of ((Int, Int), Matrix), where the (Int, Int) is the index of the block, and Matrix is the sub-matrix at the given index with size rowsPerBlock x colsPerBlock. 
- BlockMatrix supports methods such as add and multiply with another BlockMatrix. 
- BlockMatrix also has a helper function validate which can be used to check whether the BlockMatrix is set up properly.

In [19]:
from pyspark.mllib.linalg import Matrices
from pyspark.mllib.linalg.distributed import BlockMatrix

# Create an RDD of sub-matrix blocks.
blocks = sc.parallelize([((0, 0), Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])),
                         ((1, 0), Matrices.dense(3, 2, [7, 8, 9, 10, 11, 12]))])

# Create a BlockMatrix from an RDD of sub-matrix blocks.
mat = BlockMatrix(blocks, 3, 2)

# Get its size.
m = mat.numRows()  # 6
n = mat.numCols()  # 2

# Get the blocks as an RDD of sub-matrix blocks.
blocksRDD = mat.blocks

# Convert to a LocalMatrix.
localMat = mat.toLocalMatrix()

# Convert to an IndexedRowMatrix.
indexedRowMat = mat.toIndexedRowMatrix()

# Convert to a CoordinateMatrix.
coordinateMat = mat.toCoordinateMatrix()